# Neural network models with Tensorflow and Keras
In the previous sections you have learned how a computer can approach a regression or classification problem. In addition, we have seen, how more complex functions can be modelled using hidden layers and non-linear activation functions. Building a complex neural network model and training it (optimizing the weights for minimal loss) is a difficult task. We will take the easy way and use what others have prepared for us. Keras is a straight-forward Python package that does all the heavy lifting involved like taking care of the coding to create the model internally and the weight changes for each optimization step. Since Tensorflow 2.x, Keras is fully integrated into Tensorflow.

<p>
    <center>
        <a href="https://www.tensorflow.org/guide/keras">
            <img src="https://miro.medium.com/max/600/1*HLziSq4zU8TNCNJBuuQQVw.jpeg" height=200 style='height: 200px'>
        </a>
        (image: https://towardsdatascience.com/tensorflow-is-in-a-relationship-with-keras-introducing-tf-2-0-dcf1228f73ae)
    </center>
</p>
            
           



This tutorial will introduce how to use Keras to create neural network models. And because this is the first step for everyone who does anything with data, we will start with preprocessing:

## Data pre-processing
We will have a look what is important when preparing data for the use in neural networks. First, we will have a look at the data set we used in the previous tutorial:

In [ ]:
import pandas as pd
# import the data into a pandas DataFrame
data = pd.read_csv('circle_data.csv', index_col=0)
data.head()

The data consists of $x$ and $y$ coordinates of data points and a class label to characterize it. We should have a look at the data using matplotlib:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# plot data points with class 0
plt.plot(data[ data['class'] == 0]['x'], data[ data['class'] == 0]['y'], 'o', label='Class 0')
# plot data with class 1
plt.plot(data[ data['class'] == 1]['x'], data[ data['class'] == 1]['y'], 'o', label='Class 1')

plt.xlabel('x')
plt.ylabel('y')
plt.legend(loc=2, bbox_to_anchor=(1,1))

### Design matrix and target vector
In all machine learning frameworks, there is a convention on how to format the features and the array containing the target (in our case the class). The matrix containing the features is called *design matrix* and is often denoted as $X$. Per convention, it needs to have the dimension $(n_\mathrm{samples}, n_\mathrm{features})$. Let's inspect the value matrix from the data frame:

In [ ]:
X = data[['x','y']].values
X.shape

This looks good! The data is already in the required shape. If not, we can use for example numpy.reshape to get the data in the correct format. The *.values* is necessary, to get a numpy array from the data frame. Let's next inspect the target vector $Y$:

In [ ]:
Y = data[['class']].values
Y.shape

In this example you were lucky, because the data already comes in a convenient shape. This is certainly not always the case!

## My first Keras model
Let's start to build a simple neural network using Keras. We will build a *Sequential* model. This means, that the *layers* we want to use are stacked on top of each other sequentially. The type of *layer* where each node is connected with each node from the previous layer which we saw in the last tutorial is called a *Dense* layer. There are many different layer designs, but we will keep it simple for now. To start, we will reproduce the simple neural network from the previous tutorial:

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense

In [ ]:
simple_nn = Sequential(name='simple_nn')
# our input layer
# each input sample has two features, therefore the network will see an array with shape (2,)
# the sample dimension is not taken into account here!
simple_nn.add(Input(shape=(2,), name='input_layer'))
# we add a Dense layer to the model. We use 2 hidden neurons and a sigmoid activation function
simple_nn.add(Dense(2, activation='sigmoid', name='hidden_layer'))
# we add another Dense layer. Because it is also the last layer of the network, it is the output node
simple_nn.add(Dense(1, activation='sigmoid', name='output_layer'))
simple_nn.summary()

The *summary* method allows you to inspect how your model looks like. The names for the model and the layers are optional but can be helpful if you want to have a more understandable summary or you want to look e. g. at the value of weights of a certain layer. The model we created here corresponds to the one we saw in tutorial 2:

<p>
    <center>
        <img src='https://github.com/flome/e4_bsc_python/blob/machine_learning/4.%20Machine%20Learning/imgs/simple_neuron_complex.png?raw=true' height=300 style='height: 300px'>
    </center>
</p>

You see that the summary also shows the *trainable* weights. Counting the weights in the image, you can confirm the number 6 for the first layer and 3 for the second layer.

Keras has a lot of optimizers and loss functions already predefined. We will now prepare the model for *training*. For this, we need to choose an appropriate loss function for our problem and an optimizer to update the weights in each step. In Keras, this is called *compiling* a model. We will use the *binary crossentropy* we learned about before as loss function. As optimizer, we will use the *Adam* optimizer, which pretty much is a smarter way of doing gradient descent. We will use the default learning rate and therefore not specify it.

In [ ]:
simple_nn.compile(loss='binary_crossentropy', optimizer='adam')

Now it is time to let *Adam* adjust our weights. We will certainly not do that by clicking a 'Descend!' button. In Keras this is is done with the *fit* method. We pass it the design matrix $X$, the target vector $Y$ and the number of *epochs*. Most modern optimizers don't use all data points at one to compute the next weight update step. They use *batches* of input data. The default is 32. This means, that for every 32 data points from the data set, a gradient will be computed and the weights will be updated. One epoch is finished, as soon as all data points have been used one for the weight update. The process starts the from new:

In [ ]:
loss_history = simple_nn.fit(X, Y, epochs=100)

We see, that the loss is slowly decreasing with every epoch. Let's see, what the fit method returned:

In [ ]:
loss_history

We recieved a *callback history* object. It contains data that is stored during the optimization process. We are now interested in the loss values over the course of the training epochs:

In [ ]:
loss = loss_history.history['loss']
plt.plot(loss, '.')
plt.xlabel('epoch')
plt.ylabel('b. c. e loss')

You already know a loss curve like this from when you 'manually' trained your neural network in the first tutorial. A neural network is not fully trained until the loss curve flattens out.
We could go back and continue training, but first, we want to see, how the decision regions that we looked at in the previous tutorials look like now. We will use the Python package mlxtend for this:

In [ ]:
# if you get an error because you don't have mlxtend installed, you can install it with pip:
# !pip install mlxtend

In [ ]:
from mlxtend.plotting import plot_decision_regions

In [ ]:
Y_for_mlxtend = Y.flatten().astype(int) # plot_decision_regions needs a 1D int array
plot_decision_regions(X, Y_for_mlxtend, clf=simple_nn)
plt.xlabel('x')
plt.ylabel('y')
plt.legend(loc=2, bbox_to_anchor=(1,1))

The *capacity* of our neural network is not large enough to approximate a circle as decision boundary. Let's construct a slightly more complex neural network model:

In [ ]:
nn_number_two = Sequential(name='slightly_more_complex_nn')

nn_number_two.add(Input(shape=(2,), name='input_layer'))
nn_number_two.add(Dense(20, activation='sigmoid', name='hidden_layer'))
nn_number_two.add(Dense(1, activation='sigmoid', name='output_layer'))
nn_number_two.summary()

We can pass the *compile* method not only a loss function that it minimizes, but also *metrics*, that we want to track. Many metrics like *accuracy* are already predefined and ready to use:

In [ ]:
nn_number_two.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
loss_history = nn_number_two.fit(X, Y, epochs=100)

Now, we can also plot the accuracy and how it changes with the epochs:

In [ ]:
loss = loss_history.history['loss']
plt.plot(loss, '.')
plt.xlabel('epoch')
plt.ylabel('b. c. e loss')
plt.show()

loss = loss_history.history['accuracy']
plt.plot(loss, '.')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

In [ ]:
Y_for_mlxtend = Y.flatten().astype(int) # plot_decision_regions needs a 1D int array
plot_decision_regions(X, Y_for_mlxtend, clf=nn_number_two)
plt.xlabel('x')
plt.ylabel('y')
plt.legend(loc=2, bbox_to_anchor=(1,1))

As soon as the network *learns* to form a circular decision region around the inner data points, the loss dramatically decreases. The number of hidden nodes we changed is just one of many parameters we can tune during the search for a goof neural network model. These parameters we can adjust, like the number of hidden neurons, the number of hidden layers, the optimizer, the loss function, the learning rate, .... are called *hyperparameters*. Most of the time training a neural network is typically spent on the search for optimal hyperparameters for a given problem.

Experiment a bit further with the network above. Always have a look at the *loss curves* of your training, they are one of the most insightful sources of information for the hyperparameter tuning.